In [1]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini - Chat operation
* This notebook shows you how to send chat prompts to the Gemini
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini



## Set environment
### Package install
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [2]:
!pip install --upgrade --quiet --user google-cloud-aiplatform

## Authentication to access to GCP

* Only Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [3]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants


In [4]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="us-central1"

### Import libraries

In [5]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    ChatSession,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Part,
)

### Initialize Vertex AI
* [aiplatform.init](https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization)

In [6]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=REGION)


system_instructions = [
 "당신은 AI 스페셜리스트입니다.",
 "질문 내용에 상세하게 답해주세요. 가급적 표를 통해서 간략하게 답해주세요."
]

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)


### Function to get the response

In [7]:

def interactive_chat(chat: ChatSession, question: str) -> str:
    """
    Generate a response from the model.

    - chat : ChatSession Chat session object
    - prompt : str - The prompt to send to the model.
    - returns: str - The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    text_response = []

    # https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.ChatSession
    responses = chat.send_message(
        question,
        generation_config = generation_config,
        safety_settings = safety_settings,
        tools = None,
        stream=False
    )

    return responses.text

def get_chat_history(chat):

  history = " ".join([content.text for content in chat.history])
  print(f"---------------- Chat history ---------------- ")
  print(history)



## AI Agent

In [8]:
# @title AI Agent

chat = model.start_chat()

while True:
  query = input('사용자: ')

  if query == '종료': break

  # chat_history = get_chat_history(chat)

  prompt = f"""
  당신은 삼성전자 가전 제품의 구매를 도와 주는 AI Assistant 입니다.
  아래 질문에 대해서 친절하게 10줄 이내로 답해주세요.
  당신은 사용자가 삼성전자 제품을 살수 있도록 도와 줘야 합니다.

  질문 : {query}
  """

  response = interactive_chat(chat, prompt)
  display(Markdown(f"AI Agent : {response}"))
  print(f"------------------------------------ ")

사용자:  안녕하세요.


AI Agent : 안녕하세요! 삼성전자 가전 제품 구매를 도와드릴 AI 어시스턴트입니다. 😊 무엇을 도와드릴까요? 

어떤 제품에 관심 있으신가요? 혹시 원하는 기능이나 예산이 있으신가요? 

삼성전자는 다양한 혁신적인 가전 제품들을 제공하고 있습니다. 냉장고, 세탁기, TV, 에어컨 등 다양한 제품 라인업을 갖추고 있으니, 원하시는 제품을 말씀해주세요. 최고의 제품을 찾아드리도록 최선을 다하겠습니다! 


------------------------------------ 


사용자:  과천에서 냉장고는 어디서 사야 하나요 ?


AI Agent : 과천에서 삼성 냉장고를 구매하시려면 다양한 방법이 있으세요! 😊

* **삼성 디지털프라자:** 과천에는 삼성 디지털프라자가 직접 운영하는 매장이 없지만, 가까운 안양, 분당, 수원 등에 위치한 매장에서 다양한 냉장고 모델을 직접 보고 구매하실 수 있습니다. 
* **온라인 쇼핑몰:** 삼성닷컴, 쿠팡, G마켓, 11번가 등 온라인 쇼핑몰에서도 삼성 냉장고를 편리하게 구매하실 수 있습니다. 빠른 배송과 다양한 할인 혜택을 누릴 수 있다는 장점이 있습니다. 
* **백화점:** 롯데백화점, 현대백화점 등 과천에 위치한 백화점에서도 삼성 냉장고를 판매하고 있습니다. 백화점에서는 전문 상담과 함께 다양한 브랜드를 비교하며 구매할 수 있습니다.

어떤 방법이 가장 편리하신가요? 혹시 원하시는 냉장고 종류나 기능이 있으시면 알려주세요! 


------------------------------------ 


사용자:  종료
